In [1]:
import os
from os import path
    
import pandas as pd
import urllib.request
import zipfile

In [2]:
if not os.path.exists("../data/"):
    os.mkdir("../data/")
    
if not os.path.exists("../output/"):
    os.mkdir("../output/")

In [3]:
#Set resource URLs for downloading
ffiec_census_url = "https://www.ffiec.gov/Census/Census_Flat_Files/CENSUS2019.zip"
census_delineation_url = "https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2018/delineation-files/list1_Sep_2018.xls"

In [4]:
#Download Census delineation file to get MSA/MD names
urllib.request.urlretrieve(census_delineation_url, "../data/census_delineation.xls")

data_xls = pd.read_excel('../data/census_delineation.xls', 'List 1', index_col=None)
data_xls.to_csv("../data/census_delineation.csv", encoding='utf-8', index=False)
census_df = pd.read_csv("../data/census_delineation.csv", skiprows=2, dtype=object)
census_df["full_county_fips"] = census_df.apply(lambda x: x["FIPS State Code"] + x["FIPS County Code"], axis=1)

#get single name for each Metropolitan or Micropolitan statistical area using CSA over CBSA Title
census_df["MSA/MD Name"] = census_df.apply(lambda x: 
                                   x["CSA Title"] if pd.notnull(x["CSA Title"]) else x["CBSA Title"], axis=1)
census_df = census_df[["CBSA Code", "full_county_fips", "MSA/MD Name"]]
census_df.head(1)


,CBSA Code,full_county_fips,MSA/MD Name
0,10100,46013,"Aberdeen, SD"


In [5]:
#Download ZIP archive from FFIEC Census site and extract it
urllib.request.urlretrieve(ffiec_census_url, "../data/ffiec_census_2019.zip")
with zipfile.ZipFile("../data/ffiec_census_2019.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/")

In [6]:
#load FFIEC Census file and extract relevant fields and resave them
#MFI = median family income
ffiec_census_df = pd.read_csv("../data/Census2019.csv", header=None, dtype=object)
col_names = {0:"hmda_year", 
             1:"msa_md", 
             2:"state_fips", 
             3:"county_fips", 
             4:"census_tract", 
             6:"small_county_flag",
             12:"tract_to_msa_mfi_pct",
             13:"ffiec_mfi",
             14:"total_persons", 
             20:"minority_population_pct", 
             580:"tract_mfi",
             899:"total_1_4_family_units",
             915:"owner_occupied_units", 
             952:"median_housing_age"}

#field order for platform:
#0, 1, 2, 3, 4, 13, 14, 20, 915, 899, 580, 12, 952, 6, msamd_name

#get MSA, state, county, tract codes
ffiec_census_df = ffiec_census_df.iloc[:, [0,1,2,3,4,13,14,20,915,899,580,12,952,6]].copy()
ffiec_census_df.rename(columns=col_names, inplace=True)
print(len(ffiec_census_df))
ffiec_census_df.head()


75883


,hmda_year,msa_md,state_fips,county_fips,census_tract,ffiec_mfi,total_persons,minority_population_pct,owner_occupied_units,total_1_4_family_units,tract_mfi,tract_to_msa_mfi_pct,median_housing_age,small_county_flag
0,2019,33860,01,001,020100,65900,1948,12.58,507,724,72727,122.93,36,T
1,2019,33860,01,001,020200,65900,2156,59.55,433,785,48750,82.40,39,T
2,2019,33860,01,001,020300,65900,2968,25.47,828,1327,55766,94.26,39,T
3,2019,33860,01,001,020400,65900,4423,17.21,1345,1806,69114,116.82,46,T
4,2019,33860,01,001,020500,65900,10763,31.54,2255,3237,75574,127.74,14,T


In [7]:
#Join MSA/MD name to ffiec_census_df using 5 digit county FIPS
ffiec_census_df["full_county_fips"] = ffiec_census_df.apply(lambda x: x.state_fips + x.county_fips, axis=1)
ffiec_census_df = ffiec_census_df.merge(census_df, how="left", on="full_county_fips")
ffiec_census_df = ffiec_census_df[['hmda_year', 'msa_md', 'state_fips', 'county_fips', 'census_tract',
       'ffiec_mfi', 'total_persons', 'minority_population_pct',
       'owner_occupied_units', 'total_1_4_family_units', 'tract_mfi',
       'tract_to_msa_mfi_pct', 'median_housing_age', 'small_county_flag', 'MSA/MD Name']].copy()
ffiec_census_df.head()

,hmda_year,msa_md,state_fips,county_fips,census_tract,ffiec_mfi,total_persons,minority_population_pct,owner_occupied_units,total_1_4_family_units,tract_mfi,tract_to_msa_mfi_pct,median_housing_age,small_county_flag,MSA/MD Name
0,2019,33860,01,001,020100,65900,1948,12.58,507,724,72727,122.93,36,T,"Montgomery-Selma-Alexander City, AL"
1,2019,33860,01,001,020200,65900,2156,59.55,433,785,48750,82.40,39,T,"Montgomery-Selma-Alexander City, AL"
2,2019,33860,01,001,020300,65900,2968,25.47,828,1327,55766,94.26,39,T,"Montgomery-Selma-Alexander City, AL"
3,2019,33860,01,001,020400,65900,4423,17.21,1345,1806,69114,116.82,46,T,"Montgomery-Selma-Alexander City, AL"
4,2019,33860,01,001,020500,65900,10763,31.54,2255,3237,75574,127.74,14,T,"Montgomery-Selma-Alexander City, AL"


In [8]:
#set header to platform names
platform_census_header = "Collection Year|MSA/MD|State|County|Census Tract|FFIEC Median Family Income|Population|Minority Population %|Number of Owner Occupied Units |Number of 1 to 4 Family Units |Tract MFI|Tract to MSA Income %|Median Age|Small County|MSA/MD Name"
platform_census_header = platform_census_header.split("|")
ffiec_census_df.columns = platform_census_header
ffiec_census_df.head()

,Collection Year,MSA/MD,State,County,Census Tract,FFIEC Median Family Income,Population,Minority Population %,Number of Owner Occupied Units,Number of 1 to 4 Family Units,Tract MFI,Tract to MSA Income %,Median Age,Small County,MSA/MD Name
0,2019,33860,01,001,020100,65900,1948,12.58,507,724,72727,122.93,36,T,"Montgomery-Selma-Alexander City, AL"
1,2019,33860,01,001,020200,65900,2156,59.55,433,785,48750,82.40,39,T,"Montgomery-Selma-Alexander City, AL"
2,2019,33860,01,001,020300,65900,2968,25.47,828,1327,55766,94.26,39,T,"Montgomery-Selma-Alexander City, AL"
3,2019,33860,01,001,020400,65900,4423,17.21,1345,1806,69114,116.82,46,T,"Montgomery-Selma-Alexander City, AL"
4,2019,33860,01,001,020500,65900,10763,31.54,2255,3237,75574,127.74,14,T,"Montgomery-Selma-Alexander City, AL"


In [9]:
ffiec_census_df.to_csv("../output/ffiec_census_2019.txt", sep="|", index=False)

In [104]:

import pandas as pd



In [107]:
with open("../data/census_data/99mfips.txt") as in_file:
    data = in_file.readlines()

data = data[16:]

In [109]:
data

['MSA/            ALT.    State/*         City/\n',
 'CMSA    PMSA    CMSA    County          Town\n',
 'FIPS    FIPS    FIPS    FIPS            FIPS    \n',
 'CODE    CODE    CODE    CODE    F       CODE    Metropolitan Area and Component Names\n',
 '\n',
 '0040                                            Abilene, TX MSA\n',
 '0040                    48441   1                       Taylor County\n',
 '\n',
 '0060                                            Aguadilla, PR MSA\n',
 '0060                    72003   1                       Aguada Municipio\n',
 '0060                    72005   1                       Aguadilla Municipio\n',
 '0060                    72099   2                       Moca Municipio\n',
 '\n',
 '0120                                            Albany, GA MSA\n',
 '0120                    13095   1                       Dougherty County\n',
 '0120                    13177   2                       Lee County\n',
 '\n',
 '0160                                       

In [153]:
spec_df = pd.read_csv("../schemas/ffiec_census_fwf_spec_2002.csv", dtype=object)
for col in spec_df.columns:
    spec_df[col] = spec_df[col].apply(lambda x: str(x).strip())

spec_df.to_csv("../schemas/ffiec_census_fwf_spec_2002.csv", index=False)
spec_df.dtypes
spec_df.head()

,Field Number,Element Label,Starting,Ending,Length,Type
0,1,“As of” Date,1,4,4,N
1,2,Metropolitan Statistical Area (MSA) Code,5,8,4,N
2,3,FIPS State Code,9,10,2,N
3,4,FIPS Country Code,11,13,3,N
4,5,Census tract-BNA,14,19,6,N


In [173]:
fields = {"Collection Year": 1,
                 "MSA/MD": 2, 
                 "State": 3, 
                 "County": 4, 
                 "Census Tract": 5, 
                 "FFIEC Median Family Income": 12, 
                 "Population": 13,
                 "Minority Population %": 10,
                 "Number of Owner Occupied Units": 201,
                 "Number of 1 to 4 Family Units": 208,
                 "Tract MFI": 176,
                 "Tract to MSA Income %": 11, 
                 "Median Age": 246,
                 "Small County": 7}

field_nums = fields.values()
field_nums = [num-1 for num in field_nums]
fwf_spec_2002 = pd.read_csv("../schemas/ffiec_census_fwf_spec_2002.csv")
fwf_spec_2006 = pd.read_csv("../schemas/ffiec_census_fwf_spec_2006.csv")
year="2002"
data_path="../data/census_data/"
census_data = pd.read_fwf(data_path + "census_data_{year}_cleaned.dat".format(year=year), 
                            widths=fwf_spec_2006["Length"], 
                            header=None, 
                            dtype=object)
census_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212
0,2002,00404,84,410,101001,T,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,00404,84,410,102001,T,NaN,NaN,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,00404,84,410,103001,T,NaN,NaN,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,00404,84,410,104001,T,NaN,NaN,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,00404,84,410,105001,T,NaN,NaN,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
fwf_spec_2002.dtypes

Field Number      int64
Element Label    object
Starting          int64
Ending            int64
Length            int64
Type             object
dtype: object

In [172]:
fwf_spec_2006.dtypes

Field Number      int64
Element Label    object
Starting          int64
Ending            int64
Length            int64
Type             object
dtype: object

In [143]:
with open("../data/census_data/census_data_2002.dat", "r") as infile:
    lines = infile.readlines()

with open("../data/census_data/census_data_2002_cleaned.dat", "wb") as outfile:
    for line in lines:
        line = line.encode("utf-8")
        outfile.write(line)


In [140]:
repr(lines[0].strip())

"'20020040484410101001T  007.75110.69000412000000509500000828000013890000253900002556000003950000470000000129000000130000009200000000000000720000000000000000000000000000008900000161000005920000006000002641000012720000006200000468000050950000002100000000000000000000000000000000000000000000000000000000000019660000000000000000000000000000000000000018000011590000005400001033000000000000103000000084000011250000013800002189000021580000019900000129000001000000010100000040000000580000006000000052000000790000005100000051000000480000005100000027000000710000002400000060000000210000002600000023000000590000003800000010000000110000000000022861000001860000011600000177000002050000022400000243000001030000003800000021000000000000000000000005000000000000000500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000130000000000000013000000000000000000000000000000050000000000000000000000000000001300000006000000050000000000000011000000000000000000000000000

In [132]:
repr(lines[0].strip()[0])

"'2'"